# **Machine Learning (ML) For Drug Discovery - Part 1**
This is a complete technology crash course, where you will get a fair idea that how ML works, and how can you use it for a project of Drug Discovery. I have tried my level best to keep things simple, and you do not feel any need to code, therefore, I must say, it suits biologist. Here, I want to acknowledge and refer materials of Dr. Chanin Nantasenamat an Associate professor in Bioinformatics. These materials are compilled by **Dr. Ashfaq Ahmad**.

## **ChEMBL Database**

(https://www.ebi.ac.uk/chembl/)

ChEMBL is a manually curated database of bioactive molecules with drug-like properties. It brings together chemicals, their bioactivity and genomic data to aid the translation of genomic information into effective new drugs.

(**Citation:** *Nucleic Acids Res. 2019; 47(D1):D930-D940. doi: 10.1093/nar/gky1075*)

I will train you on the use and importance of ChEMBL database during this part of training. I will also teach you how can we pick and process data from the ChEMBL database. 

You will be required to execute the cells by clicking the ***Play*** button, with certain changes as per your project details.


### **Step 1 - Installing and Import Required Libraries**

Running the following cell will install ChEMBL webresources for the project to initiate.
Citation: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4489283/

In [ ]:
! pip install chembl_webresource_client

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

### **Step 2 - Picking your desired Target from ChEMBL**

Here we will launch search for the desired target protein or enzyme. To overcome any possibility of a potential mistake, it is highly advised also to confirm you ChEMBL ID from the direct database (ChEMBL) search.

In [ ]:
# As an example I will use P2Y purinoceptor 1 protein (UniProt Accession P47900)
target = new_client.target
target_query = target.search('CHEMBL4315')
targets = pd.DataFrame.from_dict(target_query)
targets

### **Step 3 - Selection and Retrieval Bioactivity data**

In case you are hit with multiple data lines, then you need to choose the desired data, using the index number below in the cell.

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

If you carefully look to the data columns, you will find variety of variable. Here we are interested to perform analysis for Antagonist, therefore, I will go with IC50 column. Please read about IC50 values incase you are not aware.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df = pd.DataFrame.from_dict(res)

In [ ]:
df

Lets play with the above data that contains too many rows and columns. By running the below command, you will only see the first four data lines.

In [ ]:
df.head(4)

The standard_type column in the data contains IC50. If you want to validate, so better to search for the unique features in the standard_type

In [ ]:
df.standard_type.unique()

Now, here I want you to save your data in the Google Drive. For ease, you need first to connect your gdrive to this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Once the drive is mounted, please make a new folder with a name data where you can save your work directly

In [ ]:
! ls /content/gdrive/MyDrive/

In [ ]:
! mkdir "/content/gdrive/MyDrive/Colab Notebooks/data"

Now we are now in a position to save our data in a CSV file. I will use the name Bioactivity_data.csv

In [ ]:
df.to_csv('P2Y1-01-bioactivity_data_raw.csv', index=False)

In [ ]:
! cp P2Y1-01-bioactivity_data_raw.csv "/content/gdrive/MyDrive/Colab Notebooks/data"

In [ ]:
ls "/content/gdrive/MyDrive/Colab Notebooks/data"

How our data looks like, take a look if you want to

In [ ]:
! head P2Y1-01-bioactivity_data_raw.csv

### **Step 4 - Treating Missing Data**

There comes a possibility that some of the ChEMBL compounds may contain missing value for the **standared_value (IC50)** and **canonical_smiles**. Therefore, we need to remove such entries from the data.

In [ ]:
df2 = df[df.standard_value.notna()]

In [ ]:
df2 = df[df.canonical_smiles.notna()]

In [ ]:
df2

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])

In [ ]:
df2_nr

### **Step 5 - Data Preprocessing**

We will be interested to keep record of the three columns, **ChEMBL_id**, **canonical_smiles**, and **standard_value**.

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]

In [ ]:
df3

Now we need to save data frame or the df3 to the CSV file.

In [ ]:
df3.to_csv('P2Y1-bioactivity-data-02-preprocessed.csv', index=False)

### **Step 6 - Labeling compounds to different categories**

Here we will need to categorize these compounds into different catergories, based on the IC50 value or **standard_value**. For the purpose, we will classify three groups, **Active**, **Inactive**, **Intermediate**. Compounds with IC50 value less than 1000 nM will be considered active drugs, those greater than 10,000 nM will be considered inactive and IC50 value in greater than 1000 and below 10,000 will be termed as intermediate.

In [ ]:
df4 = pd.read_csv('P2Y1-bioactivity-data-02-preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)

In [ ]:
df5

Now we need to save the above dataframe or df5 to the CSV file.

In [ ]:
df5.to_csv('P2Y1_03_bioactivity_data_curated.csv', index=False)

In [ ]:
! zip P2Y1-part1-data.zip *.csv

In [ ]:
! cp P2Y1_03_bioactivity_data_curated.csv "/content/gdrive/MyDrive/Colab Notebooks/data"

In [ ]:
ls "/content/gdrive/MyDrive/Colab Notebooks/data"